In [2]:
# ========== 1) SETUP ==========
import numpy as np
import pandas as pd

# --- I/O (adjust paths if needed) ---
dividend_df = pd.read_csv("Dividend_Dataset.csv", parse_dates=["Announcement_Date"])
returns_df  = pd.read_csv("Merged_Data.csv",    parse_dates=["Date"])

# Expected columns:
# dividend_df: Announcement_Date, Ticker, [Dividend_Type, Dividend_Amount, Company_Name, ...]
# returns_df : Date, Ticker, Stock_Returns, Market_Returns

# --- Parameters (trading-day based) ---
EST_LENGTH    = 110   # use exactly 110 clean obs
BUFFER_LENGTH = 10    # buffer right before event (excluded from estimation)
EVENT_K       = 5     # event window [-5, +5]

# --- Safety: types, sort, dedupe events ---
returns_df["Date"] = pd.to_datetime(returns_df["Date"])
dividend_df["Announcement_Date"] = pd.to_datetime(dividend_df["Announcement_Date"])
returns_df = returns_df.sort_values(["Ticker","Date"]).reset_index(drop=True)
dividend_df = (dividend_df
               .sort_values(["Ticker","Announcement_Date"])
               .drop_duplicates(subset=["Ticker","Announcement_Date"])
               .reset_index(drop=True))

In [3]:
# ========== 2) ESTIMATION ONLY (STRICT) ==========
alpha_beta_rows = []
drop_rows_est = []

def log_drop_est(ticker, D, reason, detail=None):
    drop_rows_est.append({
        "Ticker": ticker,
        "Event_Date": pd.to_datetime(D),
        "Reason": reason,
        "Detail": detail
    })

need = EST_LENGTH + BUFFER_LENGTH

for _, ev in dividend_df[["Ticker","Announcement_Date"]].dropna().iterrows():
    ticker = ev["Ticker"]; D = ev["Announcement_Date"]
    firm = returns_df[returns_df["Ticker"] == ticker]
    if firm.empty:
        log_drop_est(ticker, D, "no_returns_for_ticker")
        continue

    pre_all = firm[firm["Date"] < D]
    if pre_all.shape[0] < need:
        log_drop_est(ticker, D, "insufficient_pre_event_history",
                     detail=f"Have {pre_all.shape[0]} < required {need}")
        continue

    pre = pre_all.tail(need).reset_index(drop=True)
    est = pre.iloc[:EST_LENGTH].copy()

    Ri = est["Stock_Returns"].to_numpy()
    Rm = est["Market_Returns"].to_numpy()

    # STRICT: must be completely clean
    if np.isnan(Ri).any() or np.isnan(Rm).any():
        log_drop_est(ticker, D, "nan_in_estimation_window",
                     detail=f"NaNs -> Stock_Returns:{int(np.isnan(Ri).sum())}, Market_Returns:{int(np.isnan(Rm).sum())}")
        continue

    var_m = np.var(Rm, ddof=1)
    if (not np.isfinite(var_m)) or (var_m == 0):
        log_drop_est(ticker, D, "zero_or_nan_market_variance_in_estimation_window",
                     detail=f"var_m={var_m}")
        continue

    cov_im = np.cov(Ri, Rm, ddof=1)[0, 1]
    beta  = cov_im / var_m
    alpha = Ri.mean() - beta * Rm.mean()

    # Optional R2
    fitted = alpha + beta * Rm
    ss_tot = np.sum((Ri - Ri.mean())**2)
    ss_res = np.sum((Ri - fitted)**2)
    R2 = 1 - ss_res / ss_tot if ss_tot > 0 else np.nan

    alpha_beta_rows.append({
        "Ticker": ticker,
        "Event_Date": D,
        "Alpha": float(alpha),
        "Beta": float(beta),
        "R2_optional": float(R2) if np.isfinite(R2) else np.nan,
        "Estimation_Obs": EST_LENGTH
    })

alpha_beta_df = pd.DataFrame(alpha_beta_rows).sort_values(["Ticker","Event_Date"]).reset_index(drop=True)
drop_log_est  = pd.DataFrame(drop_rows_est).sort_values(["Event_Date","Ticker"]).reset_index(drop=True)

# --- Save αβ estimates (requested) ---
alpha_beta_df.to_csv("alpha_beta_by_event.csv", index=False)

print(f"Estimated alphas/betas for {alpha_beta_df.shape[0]} events "
      f"(dropped in estimation: {drop_log_est.shape[0]}).")
print("Saved: alpha_beta_by_event.csv")

Estimated alphas/betas for 702 events (dropped in estimation: 58).
Saved: alpha_beta_by_event.csv


In [4]:
alpha_beta_df

,Ticker,Event_Date,Alpha,Beta,R2_optional,Estimation_Obs
0,ADANIENT.NS,2016-03-09,-0.000245,1.934967,0.371993,110
1,ADANIENT.NS,2017-05-24,0.004341,1.355969,0.051619,110
2,ADANIENT.NS,2018-05-10,0.000159,2.711392,0.334825,110
3,ADANIENT.NS,2019-05-29,-0.002520,1.776056,0.210455,110
4,ADANIENT.NS,2020-03-12,0.004043,1.457345,0.363193,110
...,...,...,...,...,...,...
697,WIPRO.NS,2022-03-25,-0.000004,0.951547,0.367928,110
698,WIPRO.NS,2023-01-13,-0.001342,1.042780,0.425205,110
699,WIPRO.NS,2024-01-12,-0.000001,1.289917,0.316941,110
700,WIPRO.NS,2025-01-17,0.001834,1.184604,0.374806,110


In [5]:
# ========== 3) EVENT WINDOW + AAR/CAAR (same format as before; no significance flags) ==========

panel_rows = []

for _, row in alpha_beta_df.iterrows():
    ticker = row["Ticker"]
    D      = row["Event_Date"]
    alpha  = row["Alpha"]
    beta   = row["Beta"]

    firm = returns_df[returns_df["Ticker"] == ticker]
    if firm.empty:
        continue

    # K trading days before (strictly < D) and 0..+K (>= D)
    preK  = firm[firm["Date"] < D].tail(EVENT_K)
    postK = firm[firm["Date"] >= D].head(EVENT_K + 1)

    # Require exact coverage
    if (len(preK) < EVENT_K) or (len(postK) < (EVENT_K + 1)):
        continue

    preK  = preK.sort_values("Date").assign(event_day_index=np.arange(-EVENT_K, 0, 1))
    postK = postK.sort_values("Date").assign(event_day_index=np.arange(0, EVENT_K + 1, 1))
    window = pd.concat([preK, postK], ignore_index=True)

    # Expected & abnormal returns (Market Model)
    expected = alpha + beta * window["Market_Returns"].to_numpy()
    AR       = window["Stock_Returns"].to_numpy() - expected

    panel_rows.append(pd.DataFrame({
        "Ticker": ticker,
        "event_date": D,
        "trading_date": window["Date"].to_numpy(),
        "event_day_index": window["event_day_index"].to_numpy(),
        "Market_Return": window["Market_Returns"].to_numpy(),
        "Stock_Return": window["Stock_Returns"].to_numpy(),
        "Expected_Return": expected,
        "Abnormal_Return": AR
    }))

sym_panel = (
    pd.concat(panel_rows, ignore_index=True)
    if panel_rows else
    pd.DataFrame(columns=[
        "Ticker","event_date","trading_date","event_day_index",
        "Market_Return","Stock_Return","Expected_Return","Abnormal_Return"
    ])
)

# ---------- AAR (per event day) ----------
if sym_panel.empty:
    aar_table = pd.DataFrame(columns=[
        "Event_Day_Index","Number_of_Events","Average_Abnormal_Return",
        "Standard_Deviation","Standard_Error","t_statistic"
    ])
else:
    aar = (
        sym_panel.groupby("event_day_index")["Abnormal_Return"]
                 .agg(Number_of_Events="count",
                      Average_Abnormal_Return="mean",
                      Standard_Deviation=lambda x: x.std(ddof=1))
                 .reset_index()
                 .sort_values("event_day_index")
    )
    aar["Standard_Error"] = aar["Standard_Deviation"] / np.sqrt(aar["Number_of_Events"])
    aar["t_statistic"]    = aar["Average_Abnormal_Return"] / aar["Standard_Error"]
    aar_table = aar.rename(columns={"event_day_index": "Event_Day_Index"})

# ---------- CAAR over [-K, +K] ----------
if sym_panel.empty:
    caar_summary = pd.DataFrame([{
        "Number_of_Events": 0,
        "CAAR_from_AAR_Sum": np.nan,
        "CAAR_from_Mean_of_CARs": np.nan,
        "Standard_Deviation_of_CARs": np.nan,
        "Standard_Error_of_CARs": np.nan,
        "t_statistic_for_Mean_CAR": np.nan
    }])
    car_by_event = pd.DataFrame(columns=["Ticker","event_date","CAR"])
else:
    caar_from_aar = float(aar_table["Average_Abnormal_Return"].sum())
    car_by_event = (
        sym_panel.groupby(["Ticker","event_date"], as_index=False)
                 .agg(CAR=("Abnormal_Return","sum"))
                 .sort_values(["Ticker","event_date"])
    )
    N_ev   = car_by_event.shape[0]
    caar_mean = float(car_by_event["CAR"].mean()) if N_ev > 0 else np.nan
    caar_sd   = float(car_by_event["CAR"].std(ddof=1)) if N_ev > 1 else np.nan
    caar_se   = (caar_sd / np.sqrt(N_ev)) if N_ev > 1 else np.nan
    caar_t    = (caar_mean / caar_se) if (isinstance(caar_se, float) and caar_se > 0) else np.nan

    caar_summary = pd.DataFrame([{
        "Number_of_Events": N_ev,
        "CAAR_from_AAR_Sum": caar_from_aar,
        "CAAR_from_Mean_of_CARs": caar_mean,
        "Standard_Deviation_of_CARs": caar_sd,
        "Standard_Error_of_CARs": caar_se,
        "t_statistic_for_Mean_CAR": caar_t
    }])

# ---------- OUTPUT (same style as your earlier printout) ----------
n_kept = sym_panel[["Ticker","event_date"]].drop_duplicates().shape[0] if not sym_panel.empty else 0
print(f"Built symmetric panel for [-{EVENT_K}, +{EVENT_K}] with {len(sym_panel)} rows across {n_kept} events.")

print("\nAAR table (pooled by event day):")
print(aar_table.to_string(index=False))

print("\nCAAR summary for [-{0}, +{0}] window:".format(EVENT_K))
print(caar_summary.to_string(index=False))

# ---------- SAVE EXTRA CSVs (but don't print contents) ----------
#sym_panel.to_csv(f"AR_panel_[-{EVENT_K},+{EVENT_K}].csv", index=False)
#aar_table.to_csv(f"AAR_table_[-{EVENT_K},+{EVENT_K}].csv", index=False)
#caar_summary.to_csv(f"CAAR_summary_[-{EVENT_K},+{EVENT_K}].csv", index=False)
car_by_event.to_csv(f"CAR_by_event.csv", index=False)

# ========== MERGE CAR WITH DIVIDEND DATA ==========
print("\nMERGING CAR WITH DIVIDEND DATA")
print("=" * 40)

# Load the dividend dataset 
dividend_df = pd.read_csv("Dividend_Dataset.csv", parse_dates=["Announcement_Date"])
car_df = pd.read_csv("CAR_by_event.csv", parse_dates=["event_date"])

# Rename and merge
car_df = car_df.rename(columns={'event_date': 'Announcement_Date'})
merged_df = pd.merge(
    car_df,
    dividend_df[['Announcement_Date', 'Ticker', 'Dividend_Amount', 'Dividend_Type']],
    on=['Announcement_Date', 'Ticker'],
    how='left'
)

# Rename Dividend_Amount to DPS
merged_df = merged_df.rename(columns={'Dividend_Amount': 'DPS'})

print(f"CAR events: {len(car_df)}")
print(f"Merged events: {len(merged_df)}")
print(f"Events with dividend data: {merged_df['DPS'].notnull().sum()}")
print(f"Events without dividend data: {merged_df['DPS'].isnull().sum()}")

# Save the merged dataset
merged_df.to_csv('merged_dividend_car_data.csv', index=False)
print("Saved: merged_dividend_car_data.csv")

# ========== NOW CONTINUE WITH PRICE MERGE ==========

Built symmetric panel for [-5, +5] with 7183 rows across 653 events.

AAR table (pooled by event day):
 Event_Day_Index  Number_of_Events  Average_Abnormal_Return  Standard_Deviation  Standard_Error  t_statistic
              -5               653                -0.000358            0.016632        0.000651    -0.550451
              -4               653                 0.000642            0.016596        0.000649     0.988550
              -3               653                 0.000163            0.015608        0.000611     0.267362
              -2               653                 0.000010            0.015567        0.000609     0.016326
              -1               653                -0.000391            0.017445        0.000683    -0.573344
               0               653                 0.000834            0.021052        0.000824     1.012865
               1               653                -0.000276            0.029670        0.001161    -0.237695
               2         

In [44]:
import pandas as pd

# Assuming you have both datasets loaded as DataFrames
dividend_df = pd.read_csv('Dividend_Dataset.csv')  # or your file path
car_df = pd.read_csv('CAR_by_event.csv')           # or your file path

# Convert date columns to datetime to ensure proper matching
dividend_df['Announcement_Date'] = pd.to_datetime(dividend_df['Announcement_Date'])
car_df['event_date'] = pd.to_datetime(car_df['event_date'])

# First, rename event_date to Announcement_Date in the car_df
car_df = car_df.rename(columns={'event_date': 'Announcement_Date'})

# Now merge the datasets
merged_df = pd.merge(
    car_df,
    dividend_df[['Announcement_Date', 'Ticker', 'Dividend_Amount', 'Dividend_Type']],
    on=['Announcement_Date', 'Ticker'],
    how='left'
)

# Rename Dividend_Amount to DPS
merged_df = merged_df.rename(columns={'Dividend_Amount': 'DPS'})

# Display the merged dataframe
print(merged_df.head())
print(f"\nMerged dataset shape: {merged_df.shape}")
print(f"\nColumns in merged dataset: {merged_df.columns.tolist()}")

# Check for any missing values in the merged data
print(f"\nMissing values in each column:")
print(merged_df.isnull().sum())

# If you want to save the merged dataset
merged_df.to_csv('merged_dividend_car_data.csv', index=False)

        Ticker Announcement_Date       CAR  DPS Dividend_Type
0  ADANIENT.NS        2016-03-09 -0.016032  0.4       Interim
1  ADANIENT.NS        2017-05-24 -0.108284  0.4         Final
2  ADANIENT.NS        2018-05-10 -0.039030  0.4         Final
3  ADANIENT.NS        2019-05-29 -0.000906  0.4         Final
4  ADANIENT.NS        2020-03-12 -0.135723  1.0       Interim

Merged dataset shape: (657, 5)

Columns in merged dataset: ['Ticker', 'Announcement_Date', 'CAR', 'DPS', 'Dividend_Type']

Missing values in each column:
Ticker               0
Announcement_Date    0
CAR                  0
DPS                  0
Dividend_Type        0
dtype: int64


### Dividend_Yield

In [6]:
import pandas as pd

# Load the stock prices dataset
prices_df = pd.read_csv('nifty50_stock_prices.csv')

# Convert date columns to datetime format
prices_df['Date'] = pd.to_datetime(prices_df['Date'])
merged_df['Announcement_Date'] = pd.to_datetime(merged_df['Announcement_Date'])

# Create a copy to avoid modifying the original during operations
result_df = merged_df.copy()

# Get event date's AdjClose price
event_prices = prices_df[['Date', 'Ticker', 'AdjClose']].copy()
event_prices = event_prices.rename(columns={'AdjClose': 'Event_Date_AdjClose'})

result_df = pd.merge(
    result_df,
    event_prices,
    left_on=['Announcement_Date', 'Ticker'],
    right_on=['Date', 'Ticker'],
    how='left'
).drop('Date', axis=1)

# Create a dictionary of all available trading dates for each ticker
available_dates = {}
for ticker in prices_df['Ticker'].unique():
    ticker_dates = set(prices_df[prices_df['Ticker'] == ticker]['Date'])
    available_dates[ticker] = sorted(ticker_dates)

def get_most_recent_trading_price(ticker, announcement_date):
    """Get the most recent trading day price before announcement date"""
    if ticker not in available_dates:
        return None
    
    # Get all trading dates that are BEFORE the announcement date
    prior_dates = [date for date in available_dates[ticker] if date < announcement_date]
    
    if not prior_dates:
        return None
    
    # Get the most recent trading date
    most_recent_date = max(prior_dates)
    
    # Get the price for that date
    return prices_df[(prices_df['Ticker'] == ticker) & 
                    (prices_df['Date'] == most_recent_date)]['AdjClose'].values[0]

# Get previous trading day's AdjClose price
result_df['Prior_Day_AdjClose'] = result_df.apply(
    lambda row: get_most_recent_trading_price(row['Ticker'], row['Announcement_Date']),
    axis=1
)

# Calculate Dividend Yield (DPS / Prior Trading Day AdjClose Price)
result_df['Dividend_Yield'] = result_df['DPS'] / result_df['Prior_Day_AdjClose']

# Display the results
print("Final DataFrame Shape:", result_df.shape)
print("\nColumns in final dataset:")
print(result_df.columns.tolist())
print(f"\nNew columns added: Event_Date_AdjClose, Prior_Day_AdjClose, Dividend_Yield")
print("\nFirst few rows:")
print(result_df.head())

# Check for any missing values
print(f"\nMissing values in new columns:")
print(result_df[['Event_Date_AdjClose', 'Prior_Day_AdjClose', 'Dividend_Yield']].isnull().sum())

# Show some examples to verify the logic
print("\nSample verification (showing announcement date and most recent trading date):")
sample = result_df.head(3).copy()
for _, row in sample.iterrows():
    ticker = row['Ticker']
    ann_date = row['Announcement_Date']
    recent_dates = [d for d in available_dates[ticker] if d < ann_date]
    most_recent = max(recent_dates) if recent_dates else None
    print(f"Ticker: {ticker}, Announcement: {ann_date.date()}, Most Recent Trading: {most_recent.date() if most_recent else 'None'}")
merged_df.to_csv('merged_dividend_car_data.csv', index=False)

Final DataFrame Shape: (657, 8)

Columns in final dataset:
['Ticker', 'Announcement_Date', 'CAR', 'DPS', 'Dividend_Type', 'Event_Date_AdjClose', 'Prior_Day_AdjClose', 'Dividend_Yield']

New columns added: Event_Date_AdjClose, Prior_Day_AdjClose, Dividend_Yield

First few rows:
        Ticker Announcement_Date       CAR  DPS Dividend_Type  \
0  ADANIENT.NS        2016-03-09 -0.016032  0.4       Interim   
1  ADANIENT.NS        2017-05-24 -0.108284  0.4         Final   
2  ADANIENT.NS        2018-05-10 -0.039030  0.4         Final   
3  ADANIENT.NS        2019-05-29 -0.000906  0.4         Final   
4  ADANIENT.NS        2020-03-12 -0.135723  1.0       Interim   

   Event_Date_AdjClose  Prior_Day_AdjClose  Dividend_Yield  
0            35.040146           33.800018        0.011834  
1            60.596619           60.356586        0.006627  
2            79.972527           81.487999        0.004909  
3           153.172226          155.640350        0.002570  
4           160.570099    

In [9]:
result_df.to_csv('dividend_yield_analysis.csv', index=False)

### Dividend_Change_Pct

In [13]:
# Calculate dividend change for same dividend type
result_df['DPS_Change'] = result_df.groupby(['Ticker', 'Dividend_Type'])['DPS'].diff()

# Save to CSV
result_df.to_csv('dividend_yield_analysis_final.csv', index=False)

print("Final dataset created!")
print("Columns available:")
print(result_df.columns.tolist())
print(f"\nDPS_Change statistics:")
print(result_df['DPS_Change'].describe())

Final dataset created!
Columns available:
['Ticker', 'Announcement_Date', 'CAR', 'DPS', 'Dividend_Type', 'Event_Date_AdjClose', 'Prior_Day_AdjClose', 'Dividend_Yield', 'DPS_Change', 'DPS_Pct_Change', 'Dividend_Direction_Text', 'Dividend_Direction_Num']

DPS_Change statistics:
count    577.000000
mean       1.080364
std       18.244433
min     -133.000000
25%        0.000000
50%        0.250000
75%        2.400000
max      157.000000
Name: DPS_Change, dtype: float64


In [14]:
# Calculate dividend change for same dividend type
result_df['DPS_Change'] = result_df.groupby(['Ticker', 'Dividend_Type'])['DPS'].diff()

# Calculate market percentage change (assuming you have a 'Price' column)
result_df['DPS_Pct_Change'] = result_df.groupby('Ticker')['DPS'].pct_change()

# Save dataset without direction columns
result_df.to_csv('dividend_analysis_complete.csv', index=False)

print("Complete dataset created!")
print(f"Total rows: {len(result_df)}")
print("\nColumns available:")
print(result_df.columns.tolist())
print(f"\nDPS_Change statistics:")
print(result_df['DPS_Change'].describe())

Complete dataset created!
Total rows: 657

Columns available:
['Ticker', 'Announcement_Date', 'CAR', 'DPS', 'Dividend_Type', 'Event_Date_AdjClose', 'Prior_Day_AdjClose', 'Dividend_Yield', 'DPS_Change', 'DPS_Pct_Change', 'Dividend_Direction_Text', 'Dividend_Direction_Num']

DPS_Change statistics:
count    577.000000
mean       1.080364
std       18.244433
min     -133.000000
25%        0.000000
50%        0.250000
75%        2.400000
max      157.000000
Name: DPS_Change, dtype: float64


In [49]:
div_change_df.to_csv('dividend_analysis_complete.csv', index=False)

### Combined all independent variable and dependent for Regression

In [114]:
# 1. Drop duplicate columns
if 'Dividend_Direction' in regression_df.columns:
    regression_df = regression_df.drop('Dividend_Direction', axis=1)

# 2. Drop any FY_Year_x, FY_Year_y columns
fy_columns_to_drop = ['FY_Year_x', 'FY_Year_y', 'FY_Year_mcap', 'ROA_FY_Year', 'MCap_FY_Year',  'Company_Name_x', 'Company_Name_y','Market_Cap_Cr_mcap', 'Log_Market_Cap_Cr','Market_Pct_Change']
for col in fy_columns_to_drop:
    if col in regression_df.columns:
        regression_df = regression_df.drop(col, axis=1)

# 3. Rename Relevant_FY_Year to FY_Year
if 'Relevant_FY_Year' in regression_df.columns:
    regression_df = regression_df.rename(columns={'Relevant_FY_Year': 'FY_Year'})

# 4. Rearrange columns (simplified version)
essential_columns = [
    'Announcement_Date', 'Company_Name', 'Ticker', 'FY_Year', 'Dividend_Type',
    'DPS', 'DPS_Change', 'DPS_Pct_Change', 'Dividend_Direction_Text', 'Dividend_Direction_Num',
    'Event_Date_AdjClose', 'Prior_Day_AdjClose', 'Dividend_Yield',
    'ROA', 'Market_Cap_Cr', 'CAR'
]

# Keep only columns that exist
existing_columns = [col for col in essential_columns if col in regression_df.columns]
other_columns = [col for col in regression_df.columns if col not in existing_columns]
regression_df = regression_df[existing_columns + other_columns]

# 5. Save clean dataset
regression_df.to_csv('REGRESSION_DATASET.csv', index=False)

print("Duplicate columns removed!")
print("Final columns:", regression_df.columns.tolist())

Duplicate columns removed!
Final columns: ['Announcement_Date', 'Ticker', 'Dividend_Type', 'DPS', 'DPS_Change', 'DPS_Pct_Change', 'Dividend_Direction_Text', 'Dividend_Direction_Num', 'Event_Date_AdjClose', 'Prior_Day_AdjClose', 'Dividend_Yield', 'CAR']
